In [1]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from openai import OpenAI
from GoogleNews import GoogleNews

In [2]:
client = OpenAI()

In [3]:
googlenews = GoogleNews(period="1d")

In [4]:
topic = "CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pWVXlnQVAB"

In [5]:
googlenews.set_topic(topic)

googlenews.get_news()
news = googlenews.result()
headlines = [obj["title"] for obj in news]

In [9]:
with open("test_headlines.txt", "w") as f:
    f.write("\n".join(headlines))

In [6]:
system_message = "You are a summarization tool whose goal is to summarize all the information provided. You want to be as specific as possible in your summarizations. Your input is todays financial news headlines. Return the summary of todays news in the format of a bullet point list. Each bullet should begin with the '-' character."

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": "|".join(headlines)},
    ],
    model="gpt-4o-mini",
)

summarized_headlines = chat_completion.choices[0].message.content
bullet_points = summarized_headlines.split("\n")

In [8]:
def send_email(sender_email, sender_password, recipient_email, subject, body):
    # Create a multipart message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the body to the message
    msg.attach(MIMEText(body, 'plain'))

    # try:
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender_email, sender_password)
        smtp_server.sendmail(sender_email, recipient_email, msg.as_string())
        print(f"Email sent successfully to {recipient_email}")
    # except Exception as e:
    #     print(f"Error sending to {recipient_email}: {e}")

In [14]:
recipients = ["kaanan99@gmail.com"]

subject = "Financial News letter from 10/15/2024"

body = "Hello!\nHere are some highlights from todays financial news:\n"

for bullet in bullet_points:
    body += "\n\t" + bullet + "\n"

sender_email = os.getenv("EMAIL_USERNAME")
sender_password = os.getenv("EMAIL_PASSWORD")

In [ ]:
for receptient in recipients:
    send_email(sender_email, sender_password, receptient, subject, body)